---

### **Chapter 5-37**

#### **내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라.**


- 딕셔너리, 긴 튜플, 다른 내장 타입이 복잡하게 내포된 데이터를 값으로 사용하는 딕셔너리를 만들지 말라.


In [2]:
# 딕셔너리 안에 딕셔너리, 그 안에 또 리스트가 들어가는 구조(Deeply nested)는 코드를 읽기 어렵게 함

# 피해야 할 구조: 딕셔너리 안의 딕셔너리 안의 리스트
# data['기관']['부서']['인원'][0][2] <- '2'가 무엇을 의미하는지 알기 어려움
organization_data = {
    'group_a': {
        'dept_01': {
            'members': [('User1', 20, 'Level_A'), ('User2', 25, 'Level_B')]
        }
    }
}

print(organization_data)

{'group_a': {'dept_01': {'members': [('User1', 20, 'Level_A'), ('User2', 25, 'Level_B')]}}}


- 완전한 클래스가 제공하는 유연성이 필요하지 않고 가벼운 불변 데이터 컨테이너가 필요하다면 `namedtuple`을 사용하라


In [3]:
# namedtuple : 데이터의 의미를 명확히 하면서도 가볍게 유지하고 싶을 때 사용합니다.

from collections import namedtuple

# 각 필드에 이름을 부여하여 가독성 확보
Member = namedtuple('Member', ['id', 'age', 'rank'])

new_member = Member(id='User1', age=20, rank='Level_A')
print(new_member.id)   # 인덱스 대신 이름(.id)으로 접근 가능
print(new_member.rank) # 훨씬 직관적임

User1
Level_A


- 내부 상태를 표현하는 딕셔너리가 복잡해지면 이 데이터를 관리하는 코드를 여러 클래스로 나눠서 재작성하라.

In [4]:
# 상태 관리 로직이 복잡해지면 각 계층을 클래스로 나누는 것이 유지보수에 유리함.

class RankRecord:
    def __init__(self):
        self._history = []
    def add_record(self, score):
        self._history.append(score)

class MemberManager:
    def __init__(self):
        self._members = {}
    def get_member(self, name):
        if name not in self._members:
            self._members[name] = RankRecord()
        return self._members[name]

---

### **Chapter 5-38**

#### **간단한 인터페이스의 경우 클래스 대신 함수를 받아라.**


- 파이썬의 여러 컴포넌트 사이에 간단한 인터페이스가 필요할 때는 클래스를 정의하고 인스턴스화하는 대신 간단히 함수를 사용할 수 있음.
- 파이썬 함수나 메서드는 일급 시민이다. 따라서 (다른타입의 값과 마찬가지로) 함수나 함수 참조를 식에 사용할 수 있다.



In [ ]:
# 파이썬의 함수는 객체(일급 시민)이므로,
# API가 '실행 가능한 동작'을 요구할 때 굳이 복잡한 클래스를 만들 필요 없이 함수 자체를 넘기면 됨.

names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']

# 굳이 길이를 구하는 클래스를 정의할 필요가 없음
# def get_len(name): return len(name) 처럼 함수만 정의하면 됨

# len 함수 자체를 인자로 전달 (일급 시민)
names.sort(key=len) 

print(names)
# 결과: ['Plato', 'Socrates', 'Aristotle', 'Archimedes']

['Plato', 'Socrates', 'Aristotle', 'Archimedes']



- `__call__`특별 메서드를 사용하면 클래스의 인스턴스인 객체를 일반 파이선 함수처럼 호출할 수 있다. 


In [ ]:
# 클래스의 인스턴스이지만, 문법적으로는 함수처럼 ()를 붙여 실행할 수 있게 함.

class BetterRepeater:
    def __init__(self, message):
        self.message = message
    
    def __call__(self, times):      # 이 메서드 덕분에 인스턴스를 함수처럼 호출 가능
        return (self.message + " ") * times

# 인스턴스 생성
repeat_hello = BetterRepeater("Hello")

# 함수처럼 호출! (실제로는 repeat_hello.__call__(3)이 실행됨)
print(repeat_hello(3)) 
# 결과: Hello Hello Hello

- 상태를 유지하기 위한 함수가 필요한 경우에는 상태가 있는 클로저를 정의하는 대신 `__call__`메서드가 있는 클래스를 정의할지 고려해보라


In [ ]:
# 클로저
def make_counter():
    count = 0  # 상태(State)
    
    def counter():
        nonlocal count # 상태를 변경하기 위해 nonlocal 선언 필요
        count += 1
        return count
        
    return counter

my_counter = make_counter()
print(my_counter()) # 1
print(my_counter()) # 2
# 상태(count)를 밖에서 확인하거나 초기화하기 어려움

1
2


In [ ]:
# __call__ 메서드
class CountMissing:
    def __init__(self):
        self.added = 0  # 상태(State)가 명확함
        
    def __call__(self):
        self.added += 1
        return 0
        
# 사용 예시: defaultdict의 기본값 생성기로 사용
from collections import defaultdict

counter = CountMissing()
current = {'green': 12, 'blue': 3}
increments = [('red', 5), ('blue', 17), ('orange', 9)]
result = defaultdict(counter, current) # 객체(counter)를 함수처럼 전달

for key, amount in increments:
    result[key] += amount

# 결과 확인: 함수처럼 작동했지만, 상태(added)를 나중에 조회 가능
print(f"새로 추가된 키의 개수: {counter.added}") 
# 결과: 새로 추가된 키의 개수: 2 ('red', 'orange'가 없었으므로 __call__이 2번 호출됨)

새로 추가된 키의 개수: 2


---

### **Chapter 5-39**

#### **객체를 제너릭하게 구성하려면 `@classmethod`를 통한 다형성을 활용하라.**


- 파이썬의 클래스에는 생성자가 `__init__` 메서드 뿐이다.
- `@classmetod`를 사용하면 클래스에 다른 생성자를 정의할 수 있다.
- 클래스 메서드 다형성을 활용하면 여러 구체적인 하위 클래스의 객체를 만들고 연결하는 제너릭한 방법 제공이 가능하다.


---

### **Chapter 5-40**

#### **super로 부모 클래스를 초기화하라.**


- 파이썬은 표준 메서드 결정 순서(MRO)를 활용해 상위 클래스 초기화 순서와 다이아몬드 상속 문제를 해결함
- 부모 클래스를 초기화 할 때는 super내장 함수를 아무 인자 없이 호출하라. super를 아무 인자 없이 호출하면 파이썬 컴파일러가 자동으로 올바른 파라미터를 넣어준다.


---

### **Chapter 5-41**

#### **기능을 합성할 때는 믹스인 클래스를 사용하라.**


- 믹스인을 사용해 구현할 수 있는 기능을 인스턴스 애트리뷰트와 `__init__`을 사용하는 다중 상속을 통해 구현하지 말라.
- 믹스인 클래스가 클래스별로 특화된 기능을 필요로 한다면 인스턴스 수준에서 끼워넣을 수 있는 기능(정해진 메서드를 통해 해당 기능을 인스턴스가 제공하게 만듦)을 활용하라.
- 믹스인에는 필요에 따라 인스턴스 메서드는 물론 클래스 메서드도 포함될 수 있다.
- 믹스인을 합성하면 단순한 동작으로부터 더 복잡한 기능을 만들어낼 수 있다.



---

### **Chapter 5-42**

#### **비공개 애트리뷰트보다는 공개 애트리뷰트를 사용하라.**


- 파이썬 컴파일러는 비공개 애트리뷰트를 자식 클래스나 클래스 외부에서 사용하지 못하도록 엄격히 금지하지 않는다.
- 여러분의 내부 API에 있는 클래스의 하위 클래스를 정의하는 사람들이 여러분이 제공하는 클래스의 애트리뷰트를 사용하지 못하도록 막기보다는 애트리뷰트를 사용해 더 많은 일을 할 수 있게 허용하라.
- 비공개 애트리뷰트로 (외부나 하위 클래스의) 접근을 막으려고 시도하기보다는 보호된 필드를 사용하면서 문서에 적절한 가이드를 남겨라
- 여러분이 코드 작성을 제어할 수 없는 하위 클래스에서 이름 충돌이 일어나는 경우를 막고 싶을 때만 비공개 애트리뷰트를 사용할 것을 권장함.


---

### **Chapter 5-43**

#### **커스텀 컨테이너 타입은 `collections.abc`를 상속하라.**


- 간편하게 사용할 경우에는 파이썬 컨테이너 타입(리스트나 딕셔너리 등)을 직접 상속하라.
- 커스텀 컨테이너를 제대로 구현하려면 수많은 메서드를 구현해야 한다는 점에 주의하라.
- 커스텀 컨테이너 타입이 collection.abc에 정의된 인터페이스를 상속하면 커스텀 컨테이너 타입이 정상적으로 작동하기 위해 필요한 인터페이스와 기능을 제대로 구현하도록 보장할 수 있다.
